In [184]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
import matplotlib.pyplot as plt
from cassandra.cluster import Cluster
import requests
from fuzzywuzzy import fuzz
from collections import defaultdict
from fuzzywuzzy import process

In [45]:
pip install cassandra-driver


Note: you may need to restart the kernel to use updated packages.


Connect to Cassandra

In [142]:
cluster = Cluster(['127.0.0.1']) 
session = cluster.connect()

Create keyspace

In [47]:
keyspace_name = "data_quran"  


session.execute("CREATE KEYSPACE IF NOT EXISTS data_quran WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '1'}")

session.set_keyspace('data_quran')

use keyspace

In [48]:
session.set_keyspace('data_quran')

create table

In [134]:
create_table_query = """
    CREATE TABLE IF NOT EXISTS arabic (
        verse_number int PRIMARY KEY,
        surah int,
        ayah_id int,
        text varchar
    );
"""

try:
    session.execute(create_table_query)
    print("Table 'arabic' created successfully!")
except Exception as e:
    print("Error:", e)

Table 'arabic' created successfully!


In [138]:
create_table_query = """
    CREATE TABLE IF NOT EXISTS Tafseer (
        Name varchar,
        Surah int,
        Ayat int,
        Verse text,
        Tafseer text,
        PRIMARY KEY (Name, Surah, Ayat)
    );
"""

try:
    session.execute(create_table_query)
    print("Table 'Tafseer' created successfully!")
except Exception as e:
    print("Error: Issue creating table")
    print(e)

Table 'Tafseer' created successfully!


Extract data from csv

In [135]:
# Path to your CSV file
csv_file = r"C:\Users\ASUS\Desktop\big-data-engineering-with-python-2023\final_project\data\arabic_ayah.csv"  # Replace with your CSV file path

# Read data from CSV and insert into Cassandra
try:
    # Read CSV file into DataFrame
    df = pd.read_csv(csv_file)
    
    # Iterate over DataFrame rows and insert into Cassandra
    for row in df.itertuples(index=False):
        # Prepare and execute INSERT query
        insert_query = session.prepare("INSERT INTO arabic (verse_number, surah, ayah_id, text) VALUES (?, ?, ?, ?)")
        session.execute(insert_query, (row.verse_number, row.surah, row.ayah_id, row.text))
        
    print("Data inserted successfully!")
except Exception as e:
    print("Error: Inserting Rows")
    print(e)


Data inserted successfully!


In [140]:



file_path = r"C:\Users\ASUS\Desktop\big-data-engineering-with-python-2023\final_project\data\Quran_English_with_Tafseer.csv"

try:
    with open(file_path, 'r', encoding='utf-8') as file:
        next(file) 
        for line in file:
            parts = line.strip().split(',')  # Assuming data is CSV
            name = parts[0]
            surah = int(parts[1])
            ayat = int(parts[2])
            verse = parts[3]
            tafseer = parts[4]
            # Prepare and execute INSERT query
            insert_query = session.prepare("INSERT INTO Tafseer (Name, Surah, Ayat, Verse, Tafseer) VALUES (?, ?, ?, ?, ?)")
            session.execute(insert_query, (name, surah, ayat, verse, tafseer))
    print("Data inserted successfully!")
except Exception as e:
    print("Error: Inserting Data")
    print(e)

# Close connection
cluster.shutdown()


Data inserted successfully!


Show Al-quran data 

In [136]:
keyspace_name = "data_quran"
session.set_keyspace(keyspace_name)


table_name = "arabic"


rows = session.execute(f"SELECT * FROM {table_name} LIMIT 1")


for row in rows:
    print(row)  # 

Row(verse_number=4317, ayah_id=45, surah=42, text='وَتَرَىٰهُمۡ يُعۡرَضُونَ عَلَيۡهَا خَٰشِعِينَ مِنَ ٱلذُّلِّ يَنظُرُونَ مِن طَرۡفٍ خَفِيّٖۗ وَقَالَ ٱلَّذِينَ ءَامَنُوٓاْ إِنَّ ٱلۡخَٰسِرِينَ ٱلَّذِينَ خَسِرُوٓاْ أَنفُسَهُمۡ وَأَهۡلِيهِمۡ يَوۡمَ ٱلۡقِيَٰمَةِۗ أَلَآ إِنَّ ٱلظَّـٰلِمِينَ فِي عَذَابٖ مُّقِيمٖ')


In [144]:
keyspace_name = "data_quran"
session.set_keyspace(keyspace_name)


table_name = "Tafseer"


rows = session.execute(f"SELECT * FROM {table_name} LIMIT 1")


for row in rows:
    print(row)  # 

Row(name='The Reality', surah=69, ayat=1, tafseer='The Reality the Resurrection in which is realised the truth of all that was rejected in the way of the raising from the graves the reckoning and the requital or it means the Resurrection which will manifest all of that.', verse='The Reality!')


The Question

In [161]:
query = "SELECT ayah_id, text FROM arabic"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []


search_phrase ="الٓمٓ"

occurrence_count = 0
for row in rows:
    if search_phrase in row.text:
        occurrence_count += 1


print(f"The phrase '{search_phrase}' occurs {occurrence_count} times in the database.")

The phrase 'الٓمٓ' occurs 8 times in the database.


In [167]:
query = "SELECT ayah_id, text FROM arabic"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []


search_phrase ="وَ"

occurrence_count = 0
for row in rows:
    if search_phrase in row.text:
        occurrence_count += 1


print(f"The phrase '{search_phrase}' occurs {occurrence_count} times in the database.")

The phrase 'وَ' occurs 4758 times in the database.


In [172]:
query = "SELECT ayah_id, text FROM arabic"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []


search_phrase ="ٱلَّذِينَ"

occurrence_count = 0
for row in rows:
    if search_phrase in row.text:
        occurrence_count += 1


print(f"The phrase '{search_phrase}' occurs {occurrence_count} times in the database.")

The phrase 'ٱلَّذِينَ' occurs 879 times in the database.


In [178]:

# Given length to find similar ayat lengths
given_length = 50  # Replace with the length you want to find similar ayat lengths for

# Fetch all ayat lengths from the arabic table
query = "SELECT ayah_id, text FROM arabic"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []

# Calculate the absolute differences between each ayat length and the given length
similar_ayat = []
for row in rows:
    ayah_id = row.ayah_id
    ayah_text = row.text
    ayah_length = len(ayah_text)
    difference = abs(ayah_length - given_length)
    similar_ayat.append((ayah_id, ayah_text, ayah_length, difference))

# Sort the ayat by the absolute differences
similar_ayat.sort(key=lambda x: x[3])

# Print the top 5 ayat with lengths most similar to the given length
print(f"Ayat with lengths most similar to {given_length}:")
for i, (ayah_id, ayah_text, ayah_length, difference) in enumerate(similar_ayat[:5], 1):
    print(f"{i}. Ayah ID: {ayah_id}, Length: {ayah_length}, Difference: {difference}, Text: {ayah_text}")


Ayat with lengths most similar to 50:
1. Ayah ID: 75, Length: 50, Difference: 0, Text: وَلَقَدۡ نَادَىٰنَا نُوحٞ فَلَنِعۡمَ ٱلۡمُجِيبُونَ
2. Ayah ID: 32, Length: 50, Difference: 0, Text: وَإِن كُلّٞ لَّمَّا جَمِيعٞ لَّدَيۡنَا مُحۡضَرُونَ
3. Ayah ID: 4, Length: 50, Difference: 0, Text: أَلَا يَظُنُّ أُوْلَـٰٓئِكَ أَنَّهُم مَّبۡعُوثُونَ
4. Ayah ID: 124, Length: 50, Difference: 0, Text: إِذۡ قَالَ لَهُمۡ أَخُوهُمۡ هُودٌ أَلَا تَتَّقُونَ
5. Ayah ID: 40, Length: 50, Difference: 0, Text: وَمِنَ ٱلَّيۡلِ فَسَبِّحۡهُ وَأَدۡبَٰرَ ٱلسُّجُودِ


In [ ]:
query = "SELECT surah, ayat, tafseer FROM Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []

# Define the search word
search_word = "battle"

# Perform search
occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1

# Print the result
print(f"The word '{search_word}' appears {occurrence_count} times in the 'Tafseer' table.")


The word 'battle' appears 6 times in the 'Tafseer' table.


In [165]:
query = "SELECT surah, ayat, tafseer FROM Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []

# Define the search word
search_word = "humans"

# Perform search
occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1

# Print the result
print(f"The word '{search_word}' appears {occurrence_count} times in the 'Tafseer' table.")


The word 'humans' appears 12 times in the 'Tafseer' table.


In [166]:
query = "SELECT surah, ayat, tafseer FROM Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []

# Define the search word
search_word = "Muslim"

# Perform search
occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1

# Print the result
print(f"The word '{search_word}' appears {occurrence_count} times in the 'Tafseer' table.")


The word 'Muslim' appears 12 times in the 'Tafseer' table.


In [163]:
query = "SELECT surah, ayat, tafseer FROM Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []

# Define the search word
search_word = "battle"

# Perform search
occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1

# Print the result
print(f"The word '{search_word}' appears {occurrence_count} times in the 'Tafseer' table.")


The word 'battle' appears 6 times in the 'Tafseer' table.


In [182]:
query = "SELECT surah, ayat, tafseer FROM Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []

# Define the search word
search_word = "peace"

# Perform search
occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1

# Print the result
print(f"The word '{search_word}' appears {occurrence_count} times in the 'Tafseer' table.")


The word 'peace' appears 28 times in the 'Tafseer' table.


In [188]:
query = "SELECT surah, ayat, tafseer FROM Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []

# Define the search word
search_word = "fira"

# Perform search
occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1

# Print the result
print(f"The word '{search_word}' appears {occurrence_count} times in the 'Tafseer' table.")

The word 'fira' appears 2 times in the 'Tafseer' table.


In [189]:
query = "SELECT surah, ayat, tafseer FROM Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []

# Define the search word
search_word = "pasture"

# Perform search
occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1

# Print the result
print(f"The word '{search_word}' appears {occurrence_count} times in the 'Tafseer' table.")

The word 'pasture' appears 2 times in the 'Tafseer' table.


In [191]:
query = "SELECT surah, ayat, tafseer FROM Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []

# Define the search word
search_word = " obeying"

# Perform search
occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1

# Print the result
print(f"The word '{search_word}' appears {occurrence_count} times in the 'Tafseer' table.")

The word ' obeying' appears 2 times in the 'Tafseer' table.


In [193]:
query = "SELECT surah, ayat, tafseer FROM Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []

# Define the search word
search_word = "drunkenness"

# Perform search
occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1

# Print the result
print(f"The word '{search_word}' appears {occurrence_count} times in the 'Tafseer' table.")

The word 'drunkenness' appears 1 times in the 'Tafseer' table.


In [194]:
query = "SELECT surah, ayat, tafseer FROM Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []

# Define the search word
search_word = "gambling"

# Perform search
occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1

print(f"The word '{search_word}' appears {occurrence_count} times in the 'Tafseer' table.")

The word 'gambling' appears 1 times in the 'Tafseer' table.


In [198]:
query = "SELECT surah, ayat, tafseer FROM Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []

search_word = "patience"

occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1

print(f"The word '{search_word}' appears {occurrence_count} times in the 'Tafseer' table.")

The word 'patience' appears 9 times in the 'Tafseer' table.


In [200]:
query = "SELECT surah, ayat, tafseer FROM Tafseer"
try:
    rows = session.execute(query)
except Exception as e:
    print("Error executing query:", e)
    rows = []

search_word = "prophet"

occurrence_count = 0
for row in rows:
    if search_word in row.tafseer:
        occurrence_count += 1

print(f"The word '{search_word}' appears {occurrence_count} times in the 'Tafseer' table.")

The word 'prophet' appears 51 times in the 'Tafseer' table.
